In [1]:
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
import copy

In [2]:
data=pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
y=data['SalePrice']
data=data.drop('SalePrice',axis=1)


In [4]:
df=pd.concat(objs=[data,test_data],axis=0)

In [5]:
rows,column=data.shape
for feature in df:
    if df[feature].isnull().sum()>(0.6*rows):
        del df[feature]
        print(feature)

Alley
FireplaceQu
PoolQC
Fence
MiscFeature


In [6]:
#data.isnull().sum()

In [7]:
for feature in df:
    if df[feature].dtypes=='object' and df[feature].isnull().sum()>0:
            df[feature].fillna(value=df[feature].mode()[0],inplace=True)
    else:
        if  df[feature].isnull().sum()>0:
            med=df[feature].median()
            df[feature].fillna(df[feature].mean(),inplace=True)

In [8]:
#data.isnull().sum()

In [9]:
df=pd.get_dummies(df)

In [10]:
df.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706.0,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978.0,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486.0,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216.0,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655.0,...,0,0,0,1,0,0,0,0,1,0


In [11]:
leng=len(data)
train=copy.copy(df[:leng])
test=copy.copy(df[leng:])
test.shape

(1459, 271)

In [12]:

X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.25)

In [13]:
from sklearn import tree
clf=tree.DecisionTreeRegressor()
clf.fit(X_train, y_train)
x_predicted=clf.predict(X_test)
metrics.accuracy_score(y_test,x_predicted)

0.010958904109589041

In [14]:
from sklearn import neighbors
KNN=neighbors.KNeighborsRegressor()
KNN.fit(X_train, y_train)
x_predicted=clf.predict(X_test)
metrics.accuracy_score(y_test,x_predicted)

0.010958904109589041

In [15]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train,y_train)
reg.score(X_test,y_test)

0.8146638162757964

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
rfr=RandomForestRegressor()
rfr.fit(X_train,y_train)
rfr.score(X_test,y_test)

0.8993981518562569

In [17]:
from sklearn import ensemble
clf = ensemble.GradientBoostingRegressor(n_estimators = 400, max_depth = 5, min_samples_split = 2,
          learning_rate = 0.1, loss = 'ls')
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

0.9088847939637661

In [26]:
saleprice=clf.predict(test)

In [27]:
submit=pd.read_csv('sample_submission.csv')
submission=pd.DataFrame({
    "Id":submit['Id'],
    "SalePrice": saleprice
})
submission.to_csv('submission.csv',index=False)

In [28]:
x=pd.read_csv('submission.csv')
x.head()

,Id,SalePrice
0,1461,122918.194660
1,1462,158246.614116
2,1463,182419.014933
3,1464,185936.631198
4,1465,185164.192136
